In [1]:
import pandas as pd
import psycopg2 as pg
import geopandas as gpd

from masterplan_tools.City_model.city_model import CityModel
from masterplan_tools.Provision_getter.provision_getter import ProvisionModel
from masterplan_tools.Balancer.balancer import MasterPlan
from masterplan_tools.Data_getter.data_getter import DataGetter

/Users/mvin/Desktop/masterplanning/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Создание модели данных

In [2]:
engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
service_types = ["schools", "kindergartens"]
matrix = pd.read_pickle("/Users/mvin/Desktop/masterplanning/masterplan_tools/output_data/MMG_SPB.pkl")

city_model = CityModel(
    city_name="Санкт-Петербург",
    city_crs=32636,
    city_admin_level=5,
    service_types=service_types,
    accessibility_matrix=matrix,
    city_id=1,
    engine=engine,
    from_device=True
)


In [3]:
city_model.collect_data()


/Users/mvin/Desktop/masterplanning/venv/lib/python3.10/site-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/mvin/Desktop/masterplanning/venv/lib/python3.10/site-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
100%|██████████| 152800/152800 [00:00<00:00, 319088.54it/s]
/Users/mvin/Desktop/masterplanning/venv/lib/python3.10/site-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_

Проверка на создание данных по графам сервиисов для вычисление обеспеченности


In [4]:
city_model.services_graphs

{'schools': <networkx.classes.graph.Graph at 0x29e2de950>,
 'kindergartens': <networkx.classes.graph.Graph at 0x29e2dce20>}

Вычисление обеспеченноси кварталов сервисами школа и детский сад

In [5]:
for service_type in city_model.service_types:    
    model = ProvisionModel(
    service_name=service_type,
    blocks=city_model.city_blocks,
    g=city_model.services_graphs[service_type]
    )

    if service_type == 'schools':
        school = model.run()
        
    elif service_type == 'kindergartens':
        kindergarten = model.run()

    # elif service_type == 'recreational_areas':
    #     greening = model.run()

greening = gpd.read_parquet('../masterplanning/masterplan_tools/output_data/recreational_areas.parquet')


количество кварталов c сервисом schools: 29
количество жилых кварталов: 1273
количество кварталов всего: 1609
количество кварталов c ошибкой: 0
количество кварталов c сервисом kindergartens: 11
количество жилых кварталов: 797
количество кварталов всего: 900
количество кварталов c ошибкой: 0


Проверка результатов вычисления обеспеченности детским садом для отдельного квартала

In [6]:
kindergarten.iloc[6013]

index                                                                           6013
id                                                                              6013
geometry                           POLYGON ((365716.43716688757 6624359.046022324...
provision_kindergartens                                                           85
id_kindergartens                                                                6013
population_prov_kindergartens                                                  12180
population_unprov_kindergartens                                                 2086
population                                                                     14267
Name: 6013, dtype: object

Проверка результатов вычисления обеспеченности школой для отдельного квартала

In [7]:
school.iloc[6013]

index                                                                     6013
id                                                                        6013
geometry                     POLYGON ((365716.43716688757 6624359.046022324...
provision_schools                                                          100
id_schools                                                                6013
population_prov_schools                                                  14267
population_unprov_schools                                                    0
population                                                               14267
Name: 6013, dtype: object

Добавление полигона територии, к которой будет происодить генерация требований

In [8]:
polygon = gpd.read_file('../masterplanning/masterplan_tools/output_data/polygon.geojson')
polygon

,id,geometry
0,1,"MULTIPOLYGON (((365681.911 6624984.768, 366090..."


Балансировка територии и расселение новых людей

In [17]:
block = DataGetter().balance_data(gdf=city_model.blocks_aggregated_info, polygon=polygon, school=school, kindergarten=kindergarten, greening=greening)
mp = MasterPlan(area=block['area'],
                current_living_area=block['current_living_area'],
                current_industrial_area=block['current_industrial_area'],
                current_population=block['current_population'],
                current_green_area=block['current_green_area'],
                current_unprov_schoolkids=block['population_unprov_schools'],
                current_unprov_kids=block['population_unprov_kindergartens'],
                current_unprov_green_population=block['population_unprov_recreational_areas'])
updated_block_info = mp.optimal_solution_indicators()


/Users/mvin/Desktop/masterplanning/masterplan_tools/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/masterplan_tools/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/masterplan_tools/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/masterplan_tools/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pand

{'area': 35.22888712300307,
 'population': 18668.0,
 'b': 27.221773175079168,
 'green_coef_G': 5.999999999999999,
 'living_area': 15.631273074352341,
 'schools_area': 1.3,
 'schools_capacity': 600,
 'kindergartens_area': 1.1,
 'kindergartens_capacity': 280,
 'green_area': 2.6405999999999996,
 'G_min_capacity': 4401.0,
 'G_max_capacity': 2200.5,
 'green_coef_G_capacity': 4401.0,
 'op_area': 0.013203000000000001,
 'parking1_area': 3.3271560000000004,
 'parking2_area': 3.2347349999999997,
 'block_id': '6013'}

Требования к преобразования територии

In [21]:
updated_block = {'area': 269.91357042657916,
 'population': 54041,
 'b': 24.000000000000004,
 'green_coef_G': 9.0,
 'living_area': 74.65175110000001,
 'school_area': 5.4,
 'schools_capacity': 3300,
 'kindergarten_area': 5.5,
 'kindergartens_capacity': 1400,
 'green_area': 47.0917,
 'G_min_capacity': 78486.16666666667,
 'G_max_capacity': 39243.083333333336,
 'green_coef_G_capacity': 52324.11111111112,
 'op_area': 0.075396,
 'parking1_area': 18.999792000000003,
 'parking2_area': 18.47202,
 "block_id": 6013
 }

Внесение изменения на основе требований в модель данных 

In [22]:
services_g = {}
for service_type in city_model.service_types:
   services_g[service_type] = DataGetter().prepare_graph(
                    blocks=city_model.city_blocks,
                    city_crs=city_model.city_crs,
                    service_type=service_type,
                    buildings=city_model.buildings,
                    service_gdf=city_model.services_gdfs[service_type],
                    updated_block_info=updated_block,
                    accessibility_matrix=city_model.accessibility_matrix,
                )

2400
5700


100%|██████████| 605/605 [00:02<00:00, 300.74it/s]


743
2143


100%|██████████| 691/691 [00:02<00:00, 289.20it/s]


Повторное вычисление обеспеченности кварталов сервисами

In [14]:
for service_type in city_model.service_types:    
    model = ProvisionModel(
    service_name=service_type,
    blocks=city_model.city_blocks,
    g=services_g[service_type]
    )

    if service_type == 'schools':
        school = model.run()
        
    elif service_type == 'kindergartens':
        kindergarten = model.run()

количество кварталов c сервисом schools: 29
количество жилых кварталов: 1273
количество кварталов всего: 1609
количество кварталов c ошибкой: 0
количество кварталов c сервисом kindergartens: 11
количество жилых кварталов: 797
количество кварталов всего: 900
количество кварталов c ошибкой: 0


Измененные данные по обеспеченности

In [15]:
school.iloc[6013]

index                                                                     6013
id                                                                        6013
geometry                     POLYGON ((365716.43716688757 6624359.046022324...
provision_schools                                                          100
id_schools                                                                6013
population_prov_schools                                                  18668
population_unprov_schools                                                    0
population                                                               18668
Name: 6013, dtype: object

In [16]:
kindergarten.iloc[6013]

index                                                                           6013
id                                                                              6013
geometry                           POLYGON ((365716.43716688757 6624359.046022324...
provision_kindergartens                                                           89
id_kindergartens                                                                6013
population_prov_kindergartens                                                  16770
population_unprov_kindergartens                                                 1897
population                                                                     18668
Name: 6013, dtype: object